In [1]:
import numpy as np 
import pandas as pd 
import re

In [2]:
from filter_pseudogene_class import FilterPseudoGeneParams, FilterPseudoGene

In [3]:
filter_params = FilterPseudoGeneParams()   
filter_params.save_data = True 
filter_params.query_file = 'AnaPla_step_4_result_mafftAlignment.fasta'

In [4]:
filter_OR = FilterPseudoGene(filter_params)
filter_OR.filter_genes()

In [7]:
filter_OR.query_df.mismatch.describe()

count      1272
unique        2
top       False
freq       1198
Name: mismatch, dtype: object

In [9]:
filter_OR.query_df[~filter_OR.query_df['mismatch']]

,name,sequence,mismatch
0,Human_OR2J3,----------------------------------------------...,False
1,AnaPla_JACEUM010000037.1_960689_961639_1::JACE...,----------------------------------------------...,False
2,AnaPla_JACEUM010000035.1_1529835_1530629_-1::J...,-------------------------ML-------------------...,False
3,AnaPla_JACEUM010000037.1_2837900_2838805_1::JA...,----------------------------------------------...,False
4,AnaPla_JACEUM010000036.1_1290376_1291296_-1::J...,----------------------------------------------...,False
...,...,...,...
1267,AnaPla_JACEUM010000037.1_3757518_3758420_-1::J...,----------------------------------------------...,False
1268,AnaPla_JACEUM010000036.1_388859_389782_1::JACE...,----------------------------------------------...,False
1269,AnaPla_JACEUM010000036.1_707184_708107_1::JACE...,----------------------------------------------...,False
1270,AnaPla_JACEUM010000034.1_1735327_1736265_1::JA...,----------------------------------------------...,False
